In [ ]:
import os
import pickle
import numpy as np
import os
import shutil
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import scipy.interpolate as interp
from collections import OrderedDict
from ml4h.TensorMap import TensorMap, Interpretation

import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text
#from official.nlp import optimization

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
def rec_dd():
    return defaultdict(rec_dd)
scores = rec_dd()
# # close file
# f.close()
scores = pickle.load(open("split_32_28_sentiment_scores_v2022_04_26.pkl", "rb"))

In [ ]:
standardized = rec_dd()
peaks=Counter()
peak_index = {}
peak_testimonials={}
new_size = 25
for drug in scores:
    for t in scores[drug]:
        for si, s in enumerate(scores[drug][t]):
            try:
                interpolator = interp.interp1d(np.arange(len(scores[drug][t][s])), scores[drug][t][s])
                standardized[drug][t][s] = interpolator(np.linspace(0, len(scores[drug][t][s])-1, new_size))
                if max(scores[drug][t][s]) > peaks[drug,s]:
                    peak_testimonials[drug,s] = t
                    peaks[drug,s] = max(scores[drug][t][s])
                    peak_index[drug,s] =  np.argmax(scores[drug][t][s])
            except ValueError:
                print(f'vall err {drug} {t}')


In [ ]:
for p in peaks:
    if 'sadness' in p:
        print(f'{p} has: {peaks[p]:0.3f} and testi: {peak_testimonials[p]} at index: {peak_index[p]}')

In [ ]:
pzz = []
count = []
testi = []
peak_sentence = {}
for p in peaks:
    
    if p[-1] == 'sadness':
        #print(p)
        print(f'{p} has: {peaks[p]:0.3f} and testi: {peak_testimonials[p]} pi {peak_index[p]}')
        count.append(peaks[p])
        testi.append(peak_testimonials[p])

        pzz.append(p)
print(f'max has: {max(count)} and testi: {testi[np.argmax(count)]} at sentence: {peak_index[pzz[np.argmax(count)]]} drug, feeling {pzz[np.argmax(count)]}')

In [ ]:
df = pd.read_csv(f'./split_32_all_drugs_v2022_04_22_meta_data.csv')
step_size = 32
df = df[df.text.notna()]
peak_key = pzz[np.argmax(count)]
ex = df[df.drug == peak_key[0]]
ex = ex[ex.testimonial == testi[np.argmax(count)]]
ex = ex[ex.sentence==peak_index[peak_key]]
print(ex.iloc[0].text)

In [ ]:
count = []
testi = []
pzz = []
for p in peaks:
    if 'curiosity' in p:
        #print(f'{p} has: {peaks[p]:0.3f} and testi: {peak_testimonials[p]} at index: {peak_index[p]}')
        count.append(peaks[p])
        testi.append(peak_testimonials[p])
        pzz.append(p)
print(f'max has: {max(count):.3f} and testi: {testi[np.argmax(count)]} drug, feeling {pzz[np.argmax(count)]}')
peak_key = pzz[np.argmax(count)]
ex = df[df.drug == peak_key[0]]
ex = ex[ex.testimonial == testi[np.argmax(count)]]
ex = ex[ex.sentence==peak_index[peak_key]]
print(ex.iloc[0].text)

In [ ]:
!pip install fastdtw

from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
standardized_psycho = rec_dd()
means_psycho = Counter()
stds_psycho = Counter()
new_size = 20
for drug in scores:
    psychoactive_class = df[df.drug==drug].psychoactive_class.unique()[0]
    for t in scores[drug]:
        for s in scores[drug][t]:
            try:
                interpolator = interp.interp1d(np.arange(len(scores[drug][t][s])), scores[drug][t][s])
                
                standardized_psycho[psychoactive_class][t][s] = interpolator(np.linspace(0, len(scores[drug][t][s])-1, new_size))
                means_psycho[psychoactive_class,s] += standardized_psycho[psychoactive_class][t][s]
                stds_psycho[psychoactive_class,s] += standardized_psycho[psychoactive_class][t][s]*standardized_psycho[psychoactive_class][t][s]
            except ValueError:
                print(f'vall err {drug} {t}')
    for s in standardized_psycho[psychoactive_class][t]:
        means_psycho[psychoactive_class,s] /= len(standardized_psycho[psychoactive_class])
        stds_psycho[psychoactive_class,s] /= len(standardized_psycho[psychoactive_class])
        stds_psycho[psychoactive_class,s] -= means_psycho[psychoactive_class,s]*means[psychoactive_class,s]
        stds_psycho[psychoactive_class,s] = np.sqrt(stds[psychoactive_class,s])
    

In [ ]:
standardized = rec_dd()
means = Counter()
stds = Counter()
new_size = 20
for drug in scores:
    for t in scores[drug]:
        for s in scores[drug][t]:
            try:
                interpolator = interp.interp1d(np.arange(len(scores[drug][t][s])), scores[drug][t][s])
                standardized[drug][t][s] = interpolator(np.linspace(0, len(scores[drug][t][s])-1, new_size))
                means[drug,s] += standardized[drug][t][s]
                stds[drug,s] += standardized[drug][t][s]*standardized[drug][t][s]
            except ValueError:
                print(f'vall err {drug} {t}')
    for s in scores[drug][t]:
        means[drug,s] /= len(scores[drug])
        stds[drug,s] /= len(scores[drug])
        stds[drug,s] -= means[drug,s]*means[drug,s]
        stds[drug,s] = np.sqrt(stds[drug,s])
    

In [ ]:
distances = {} 
idxs = Counter()
jdxs = Counter()

for drug1,sentiment1 in means:
#     if 'love' not in sentiment1:
#         continue
    if sentiment1 not in distances:
        distances[sentiment1] = np.zeros((len(scores),len(scores)))
    for drug2,sentiment2 in means:
        if sentiment1 != sentiment2:
            continue
        distance, path = fastdtw(means[drug1,sentiment1], 
                                 means[drug2,sentiment2], 
                                 dist=euclidean)
        if means[drug1,sentiment1].mean() > means[drug2,sentiment2].mean():
            sign = 1
        else:
            sign = -1
        distances[sentiment1][idxs[sentiment1], jdxs[sentiment1]] = distance * sign
        jdxs[sentiment1]+= 1
    idxs[sentiment1]+=1
    jdxs[sentiment1]=0
    print(f'Finished Drug {drug1} of {len(means)} ')
#print(f'Drug {distances} ')



In [ ]:
def confusion_heatmap(confusion, labels, cutoff=0.001, fmt='2d', figsize=(24, 15), 
                      title='Emotion Heatmap'):
    fig, ax = plt.subplots(figsize=figsize, dpi=300)
    ax = sb.heatmap(confusion, cmap='bwr', cbar_kws={"shrink": .8}, cbar=True, ax=ax)

    ax.set_title(title)
    ax.set_xticks(np.arange(confusion.shape[0]) + 0.5)
    ax.set_yticks(np.arange(confusion.shape[0]) + 0.5)
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([np.min(confusion), np.max(confusion)])
    cbar.set_ticklabels([f'Less \n{title.split()[0].capitalize()}', 
                         f'More \n{title.split()[0].capitalize()}'])
    ax.set_xticklabels(labels=labels, ha='right', rotation=45)
    ax.set_yticklabels(labels=labels, rotation=0)
    plt.tight_layout()
    figure_path='./confusion_heatmap.png'
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    plt.show()

In [ ]:
import seaborn as sb
confusion_heatmap(distances['love'], list(scores.keys()), fmt=".2f", figsize=(16, 10))

In [ ]:
confusion_heatmap(distances['joy'], list(scores.keys()), fmt=".2f", figsize=(16, 10),
                 title='Joy Dynamic Time Warp Distances Cross-Correlation')

In [ ]:
confusion_heatmap(distances['realization'], list(scores.keys()), fmt=".2f", figsize=(16, 10),
                 title='Realization Dynamic Time Warp Distances Cross-Correlation')

In [ ]:
confusion_heatmap(distances['sadness'], list(scores.keys()), fmt=".2f", figsize=(18, 12),
                 title='Sadness Dynamic Time Warp Distances Cross-Correlation')

In [ ]:
#distances = defaultdict(list)
for sentiment in distances:
    confusion_heatmap(distances[sentiment], 
                      list(scores.keys()), fmt=".2f", figsize=(15, 10),
                      title=f'{sentiment.capitalize()} Dynamic Time Warp Cross Drug Trajectory Distances')    
    

In [ ]:
for d in distances:
    print(f'{d} has mean {np.mean(distances[d])}')

In [ ]:
df = pd.read_csv(f'./all_drugs_v2022_04_26_meta_data.csv')

In [ ]:
df[df.drug=='lsd'].psychoactive_class.unique()[0]